Imports

In [20]:
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta
import os
#from subprocess import call
import subprocess

Parameters

In [21]:
# source filepath
sourceFilepath = '/home/julio/strides/data/activities_until_19_02_06/'
#sourceFilepath = '/home/julio/strides/data/fit_raw/'

Functions

In [22]:
#convert .FIT to .csv with the help of FIT_sdk
def convertFITToCsv(sourceFilepath, file):
    source_file = sourceFilepath+file+".FIT"
    target_file = "/home/julio/strides/data/fit2csv/"+file+".csv"
    #call(["java", "-jar", "/home/julio/strides/fit_sdk/java/FitCSVTool.jar", "-b", source_file, target_file])
    subprocess.run(["java", "-jar", "/home/julio/strides/fit_sdk/java/FitCSVTool.jar", "-b", source_file, target_file], stdout=subprocess.PIPE)

In [23]:
def secondsToUTC (dataset):
    #1989-12-31 00:00:00
    zero = datetime(1989,12,31,0,0,0)
    #print (datetime.strftime(zero,"%b %d %Y %H:%M:%S"))
    new_dataset = []
    for d in dataset:
        n = str(zero + timedelta(seconds=d))
        new_dataset.append(n)
    return new_dataset

In [24]:
def prepareCSV (filepath, file, filetype):
    print(file)
    #Read csv
    df = pd.read_csv(filepath+file+filetype)
    
    #filter for local number == 7 and type == Date
    df_filtered = df.loc[df['Local Number'] == 7]
    df_filtered = df_filtered.loc[df_filtered['Type'] == 'Data']
    
    #convert position and time
    df_filtered['Value 2'] = df_filtered['Value 2']* 180/(2**31)
    df_filtered['Units 2'] = "degrees"

    df_filtered['Value 3'] = df_filtered['Value 3']* 180/(2**31)
    df_filtered['Units 3'] = "degrees"

    df_filtered['Value 1'] = secondsToUTC(df_filtered['Value 1'])
    df_filtered['Units 1'] = "UTC"
    
    # drop unnecessary data
    toBeDropped = []
    counter = 10
    field_str = 'Field '
    value_str = 'Value '
    units_str = 'Units '

    upperBound = (len(df.columns) - 31)/3 + 10
    while counter < upperBound:
        toBeDropped.append(field_str+str(counter))
        toBeDropped.append(value_str+str(counter))
        toBeDropped.append(units_str+str(counter))
        counter = counter + 1

    toBeDropped.append("Type")
    toBeDropped.append("Local Number")
    toBeDropped.append("Message")
    toBeDropped.append("Unnamed: "+str(len(df.columns)-1))
    
    df_filtered = df_filtered.drop(columns=toBeDropped)
    
    # write csv
    df_filtered.to_csv(filepath+file+'_prepared'+filetype)

In [29]:
#Get file names in directory
def getFiles(sourceFilepath, filetype):
    files = []  
    for file in os.listdir(sourceFilepath):
        if file.upper().endswith(filetype) and not file.endswith("prepared"+filetype):
            files.append(file.split('.')[0])
    return files

Convert FIT to csv

In [30]:
files = getFiles(sourceFilepath=sourceFilepath, filetype='.FIT')
print (files)
for file in files:
    convertFITToCsv(sourceFilepath=sourceFilepath, file=file)

['2236958908', '298184996', '288481388', '1608998211', '398965578', '543200701', '2170033111', '1222068316', '474459578', '1180153693', '362495632', '1639258689', '331364485', '395292464', '1018496029', '1675992597', '356894673', '1658137692', '586381642', '1682139320', '1668341720', '1477520598', '1789862349', '1594357639', '331364462', '1393796126', '257607439', '995207294', '368892701', '774246617', '262123398', '1452241788', '928437785', '337478458', '321523801', '255658989', '480542551', '1000857742', '587344224', '372834079', '273024824', '217953876', '1380144282', '1428848414', '294663272', '456587119', '477185683', '331363803', '1557917860', '439547876', '237298950', '1664561476', '1736229674', '497464091', '267112987', '1520749381', '932288704', '1568240001', '669921300', '509154577', '338798161', '1338337970', '484914364', '864403638', '627096648', '1285234006', '1553599537', '1406232331', '965163324', '206292230', '1493085935', '1655559538', '479200288', '1428068261', '71716

Prepare csv files

In [32]:
files = getFiles(sourceFilepath=sourceFilepath, filetype='.csv')
print (files)
for file in files:
    prepareCSV (filepath='/home/julio/strides/data/fit2csv/', file=file, filetype='.csv')

[]
